In [ ]:
import numpy as np

In [ ]:
# tensorflow packages
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [19]:
train_data, validation_data, test_data = tfds.load(name='imdb_reviews',
                                                  split=('train[:60%]', 'train[60%:]', 'test'),
                                                  as_supervised=True)

In [20]:
train_ex_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [24]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1"
# embed = hub.load(module_url)
hublayer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

In [25]:
hublayer(train_ex_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.6688950e+00, -3.9186180e+00,  3.7993348e+00, -1.4097565e+00,
        -3.3214598e+00, -9.9966162e-01, -2.0233727e+00,  1.2953283e+00,
         5.0233593e+00, -1.0990871e+00, -2.0879953e+00, -1.2672297e+00,
        -1.8820252e-01, -7.7382974e-02, -2.9172704e+00,  1.7943126e+00,
         1.2375495e+00, -1.5559573e+00, -2.4811747e+00, -7.8135049e-01],
       [ 1.7287169e+00, -2.4736629e+00,  3.3280423e+00,  9.1747051e-01,
        -4.0622129e+00, -4.7915082e+00, -2.7125058e+00,  1.2155541e+00,
         3.7063696e+00, -9.5357007e-01, -1.8554710e+00,  1.2775491e+00,
        -7.6736540e-02,  6.8350559e-01, -4.8221231e+00,  2.6815715e+00,
         4.4790969e+00, -2.3602080e+00, -3.2802453e+00, -1.5463475e+00],
       [ 1.1199147e+00, -1.0038884e+00,  1.3508813e+00, -1.9272239e+00,
        -9.1316330e-01,  4.4636719e-02, -4.8363330e-03,  2.1533751e+00,
         1.0593201e+00,  4.3559158e-01, -3.0889933e+00,  9.9787104e-01,
        -1.06

In [26]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(hublayer)
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 20)                389380    
                                                                 
 dense_2 (Dense)             (None, 16)                336       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 389733 (1.49 MB)
Trainable params: 389733 (1.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [29]:
history = model.fit(train_data.shuffle(100000).batch(100),
                   epochs=25,
                   validation_data = validation_data.batch(100),
                   verbose=1)

Epoch 1/25
150/150 [==============================] - 11s 47ms/step - loss: 0.7936 - accuracy: 0.5719 - val_loss: 0.6496 - val_accuracy: 0.6350
Epoch 2/25
150/150 [==============================] - 6s 40ms/step - loss: 0.6042 - accuracy: 0.6741 - val_loss: 0.5707 - val_accuracy: 0.7067
Epoch 3/25
150/150 [==============================] - 6s 40ms/step - loss: 0.5082 - accuracy: 0.7553 - val_loss: 0.4786 - val_accuracy: 0.7723
Epoch 4/25
150/150 [==============================] - 6s 40ms/step - loss: 0.4108 - accuracy: 0.8227 - val_loss: 0.4040 - val_accuracy: 0.8219
Epoch 5/25
150/150 [==============================] - 6s 39ms/step - loss: 0.3399 - accuracy: 0.8600 - val_loss: 0.3594 - val_accuracy: 0.8442
Epoch 6/25
150/150 [==============================] - 6s 39ms/step - loss: 0.2870 - accuracy: 0.8875 - val_loss: 0.3332 - val_accuracy: 0.8584
Epoch 7/25
150/150 [==============================] - 6s 41ms/step - loss: 0.2478 - accuracy: 0.9057 - val_loss: 0.3162 - val_accuracy: 0.865

In [30]:
results = model.evaluate(test_data.batch(100), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s : %.3f" %(name, value))

250/250 - 4s - loss: 0.6252 - accuracy: 0.8418 - 4s/epoch - 16ms/step
loss : 0.625
accuracy : 0.842
